In [1]:
import numpy as np

def matrix_overview(A, sample_size=5, random_seed=42):
    """
    Prints lightweight statistics and structure overview
    for a large numeric matrix A (e.g., 5000x5000 floats).
    """
    np.random.seed(random_seed)
    rows, cols = A.shape
    
    print(f"\n📌 MATRIX SHAPE: {rows} x {cols}")
    print(f"   Total elements: {rows * cols:,}")
    print(f"   Data type: {A.dtype}\n")
    
    # ---- GLOBAL STATS (cheap) ----
    print("🔍 BASIC STATISTICS")
    print(f"   Min value:        {A.min():.6g}")
    print(f"   Max value:        {A.max():.6g}")
    print(f"   Mean value:       {A.mean():.6g}")
    print(f"   Median value:     {np.median(A):.6g}")
    print(f"   Std deviation:    {A.std():.6g}")
    print(f"   Variance:         {A.var():.6g}")
    
    # ---- DISTRIBUTION SKETCH ----
    print("\n📊 VALUE DISTRIBUTION (percentiles)")
    percentiles = [0, 1, 5, 25, 50, 75, 95, 99, 100]
    vals = np.percentile(A, percentiles)
    for p, v in zip(percentiles, vals):
        print(f"   {p:>3}% → {v:.6g}")
    
    # ---- ROW/BLOCK INSPECTION ----
    print("\n🔎 ROW & COLUMN NORMS (quick sense of magnitude)")
    row_norms = np.linalg.norm(A, axis=1)
    col_norms = np.linalg.norm(A, axis=0)
    print(f"   Mean row L2 norm:     {row_norms.mean():.6g}")
    print(f"   Max row L2 norm:      {row_norms.max():.6g}")
    print(f"   Mean column L2 norm:  {col_norms.mean():.6g}")
    print(f"   Max column L2 norm:   {col_norms.max():.6g}")
    
    # ---- SPARSITY ESTIMATE ----
    print("\n🧮 SPARSITY (how many zeros?)")
    zero_count = np.count_nonzero(A == 0)
    print(f"   Zero elements: {zero_count:,} ({zero_count / (rows*cols) * 100:.3f}%)")
    
    # ---- SAMPLE VALUES ----
    print("\n🔬 RANDOM SAMPLES FROM MATRIX")
    idx = np.random.choice(rows * cols, size=min(sample_size, rows*cols), replace=False)
    sample_vals = A.flatten()[idx]
    print(f"   {len(sample_vals)} random values: {sample_vals}")
    
    # ---- ROW/COL CORRELATION (cheap version) ----
    print("\n🔗 MEAN CORRELATION BETWEEN RANDOM ROWS")
    if rows > 10:
        sample_rows = A[np.random.choice(rows, size=5, replace=False)]
        corr = np.corrcoef(sample_rows)
        print("   Correlation matrix of 5 random rows:")
        print(np.round(corr, 3))
    else:
        print("   Matrix too small for sampling.")
    
    print("\n✅ DONE\n")

# Example usage:


In [2]:
def top_n_values_with_words(A, words, n=10):
    """
    Find top n largest values in matrix A and print value + row/col words.
    `words` is a list where words[i] corresponds to row/column i.
    """
    flat = A.ravel()
    n = min(n, flat.size)

    # argpartition → fast selection of top n values
    idx = np.argpartition(flat, -n)[-n:]
    idx_sorted = idx[np.argsort(flat[idx])[::-1]]

    print(f"\n🔥 Top {n} largest values (with word lookup):")
    results = []

    for i in idx_sorted:
        row, col = divmod(i, A.shape[1])
        val = flat[i]
        
        row_word = words[row] if row < len(words) else f"[no word {row}]"
        col_word = words[col] if col < len(words) else f"[no word {col}]"

        print(f"   {val:.6g}  at ({row}, {col}) → {row_word}  ×  {col_word}")
        results.append((val, row, col, row_word, col_word))

    return results


In [3]:
import numpy as np

def top_n_values(A, n=30):
    """
    Find the top `n` largest values in matrix A and their (row, col) locations.
    Efficient for very large matrices (uses argpartition, not full sort).
    """
    flat = A.ravel()  # view without copying
    if n > flat.size:
        n = flat.size

    # Get indices of n largest values (unordered)
    idx = np.argpartition(flat, -n)[-n:]

    # Sort those n values descending
    idx_sorted = idx[np.argsort(flat[idx])[::-1]]

    print(f"\n🔥 Top {n} largest values in the matrix:")
    results = []
    for i in idx_sorted:
        row, col = divmod(i, A.shape[1])
        val = flat[i]
        results.append((val, row, col))
        print(f"   value={val:.6g}  at (row={row}, col={col})")

    return results


In [4]:
def top_n_rows_by_sum(A, words, n=10):
    """
    Find rows with the highest total sum, print top N with index + word.
    `A` is a 2D NumPy array.
    `words` is a list mapping row index -> word (from CSV).
    """
    row_sums = A.sum(axis=1)      # Fast vectorized sum
    n = min(n, len(row_sums))     # Safety

    # Get indices of n largest sums
    idx = np.argpartition(row_sums, -n)[-n:]
    idx_sorted = idx[np.argsort(row_sums[idx])[::-1]]

    print(f"\n📌 Top {n} rows by total row sum:")
    results = []
    for i in idx_sorted:
        total = row_sums[i]
        word = words[i] if i < len(words) else f"[no word {i}]"
        print(f"   row {i:5d} → sum = {total:.6g} → {word}")
        results.append((i, total, word))

    return results

In [5]:
def load_first_column(csv_path):
    """
    Load the CSV and return a list where index -> first word in that row.
    Assumes comma-separated file, no header, first column = target word.
    """
    df = pd.read_csv(csv_path, header=None)
    return df.iloc[:, 0].astype(str).tolist()


In [6]:

A = np.load(r"C:\Users\HP\bert_experiments\matrcies\Probabilities_NGSL.npy")
for i in range(0, len(A)):
    A[i][i] = 0


In [7]:
matrix_overview(A)


📌 MATRIX SHAPE: 4552 x 4552
   Total elements: 20,720,704
   Data type: float64

🔍 BASIC STATISTICS
   Min value:        0
   Max value:        0.0197658
   Mean value:       1.08387e-07
   Median value:     2.01192e-09
   Std deviation:    7.1275e-06
   Variance:         5.08012e-11

📊 VALUE DISTRIBUTION (percentiles)
     0% → 0
     1% → 9.6416e-12
     5% → 4.69912e-11
    25% → 4.28569e-10
    50% → 2.01192e-09
    75% → 9.89049e-09
    95% → 1.17986e-07
    99% → 9.49374e-07
   100% → 0.0197658

🔎 ROW & COLUMN NORMS (quick sense of magnitude)
   Mean row L2 norm:     6.88346e-05
   Max row L2 norm:      0.0225197
   Mean column L2 norm:  7.10338e-05
   Max column L2 norm:   0.0203658

🧮 SPARSITY (how many zeros?)
   Zero elements: 4,552 (0.022%)

🔬 RANDOM SAMPLES FROM MATRIX
   5 random values: [9.19257453e-09 2.56313704e-10 1.44123554e-10 5.39996948e-10
 1.68242971e-09]

🔗 MEAN CORRELATION BETWEEN RANDOM ROWS
   Correlation matrix of 5 random rows:
[[1.    0.735 0.523 0.526 0.5

In [8]:
import pandas as pd

words = load_first_column(r"C:\Users\HP\bert_experiments\NGSL_lists\NEWEST_NGSL.csv")

# Print top 15 biggest values
top = top_n_values_with_words(A, words, n=2000)



🔥 Top 2000 largest values (with word lookup):
   0.0197658  at (3401, 2556) → the  ×  of
   0.00516149  at (1116, 3401) → be  ×  the
   0.0050928  at (1116, 3436) → be  ×  to
   0.00469597  at (3401, 2148) → the  ×  in
   0.00458008  at (887, 1116) → a  ×  be
   0.00447777  at (2556, 3401) → of  ×  the
   0.0043936  at (3401, 1116) → the  ×  be
   0.00429571  at (3436, 3401) → to  ×  the
   0.00424965  at (1116, 887) → be  ×  a
   0.00423648  at (3401, 3436) → the  ×  to
   0.00385728  at (2148, 3401) → in  ×  the
   0.00352605  at (3401, 998) → the  ×  and
   0.00351668  at (3401, 1912) → the  ×  for
   0.00348641  at (1116, 2148) → be  ×  in
   0.00345785  at (1116, 3641) → be  ×  will
   0.00336332  at (1116, 2529) → be  ×  not
   0.00307889  at (1116, 1912) → be  ×  for
   0.00294839  at (3401, 2567) → the  ×  on
   0.00286746  at (3400, 1116) → that  ×  be
   0.00271858  at (887, 2556) → a  ×  of
   0.00262181  at (2114, 1116) → I  ×  be
   0.00248137  at (2114, 3436) → I  ×  to


In [9]:
top_rows = top_n_rows_by_sum(A, words, n=300)


📌 Top 300 rows by total row sum:
   row  3401 → sum = 0.123548 → the
   row  1116 → sum = 0.121222 → be
   row   887 → sum = 0.0762635 → a
   row  3436 → sum = 0.0643535 → to
   row  2114 → sum = 0.0515683 → I
   row  3683 → sum = 0.0414648 → you
   row  3400 → sum = 0.0390296 → that
   row  2148 → sum = 0.0381484 → in
   row   998 → sum = 0.0379298 → and
   row  2556 → sum = 0.0360455 → of
   row  2050 → sum = 0.0359528 → have
   row  2051 → sum = 0.0316497 → he
   row  2227 → sum = 0.029656 → it
   row  3417 → sum = 0.0263284 → this
   row  1912 → sum = 0.0229422 → for
   row  3411 → sum = 0.0226504 → they
   row  3597 → sum = 0.0195276 → we
   row  2529 → sum = 0.0191087 → not
   row  1052 → sum = 0.0153268 → as
   row  1634 → sum = 0.0145934 → do
   row  1221 → sum = 0.0128973 → but
   row  3653 → sum = 0.0127831 → with
   row  3112 → sum = 0.0126222 → she
   row  2567 → sum = 0.0123373 → on
   row  1234 → sum = 0.0120747 → can
   row  1984 → sum = 0.0115511 → get
   row  3641 → s

In [ ]:
import numpy as np


# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues:")
print(eigenvalues)

print("\nEigenvectors (column-wise):")
print(eigenvectors)


Eigenvalues:
[ 2.05128573e-02+0.j -1.14125833e-02+0.j -5.78247943e-03+0.j ...
 -1.26750099e-09+0.j -8.81854104e-10+0.j -1.72601507e-09+0.j]

Eigenvectors (column-wise):
[[ 2.45316187e-05+0.j -2.07407685e-05+0.j -2.62487943e-05+0.j ...
  -3.23398290e-03+0.j  5.86465178e-03+0.j  3.58351669e-04+0.j]
 [ 3.27566344e-04+0.j  1.96940133e-04+0.j  2.09316312e-04+0.j ...
  -1.89795144e-02+0.j  7.71491212e-03+0.j -6.93894673e-04+0.j]
 [ 4.24992391e-05+0.j -1.91938918e-05+0.j -2.27251547e-05+0.j ...
   1.61885316e-02+0.j  1.12600002e-02+0.j -3.99398319e-03+0.j]
 ...
 [ 3.97021385e-05+0.j -3.85488991e-05+0.j -3.10916854e-05+0.j ...
   1.45179798e-03+0.j  6.94943777e-05+0.j  8.39549882e-04+0.j]
 [ 1.18987592e-04+0.j -6.60556392e-05+0.j -1.14351010e-04+0.j ...
  -1.68771156e-04+0.j -9.94317691e-04+0.j  2.98365041e-03+0.j]
 [ 1.63022919e-04+0.j  1.30494271e-04+0.j  6.28675179e-05+0.j ...
   9.87535570e-03+0.j -6.02937029e-03+0.j -5.82680248e-03+0.j]]


In [13]:
# --- Basic statistics ---
real_parts = np.real(eigenvalues)
imag_parts = np.imag(eigenvalues)

print("\n--- Eigenvalue Statistics ---")
print(f"Count: {len(eigenvalues)}")
print(f"Min (real part): {real_parts.min()}")
print(f"Max (real part): {real_parts.max()}")
print(f"Mean (real part): {real_parts.mean():.16f}")
print(f"Std dev (real part): {real_parts.std():.10f}")

if np.any(imag_parts != 0):
    print("\n(Complex components detected)")
    print(f"Mean (imag part): {imag_parts.mean():.10f}")
    print(f"Std dev (imag part): {imag_parts.std():.10f}")


--- Eigenvalue Statistics ---
Count: 4552
Min (real part): -0.011412583252419275
Max (real part): 0.020512857281127485
Mean (real part): 0.0000000000000000
Std dev (real part): 0.0003769808

(Complex components detected)
Mean (imag part): -0.0000000000
Std dev (imag part): 0.0000081663
